# Setup

In [1]:
import sys
import os
import locale

import re
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from urllib.parse import parse_qs, urlparse

pd.options.display.max_rows = 20000
pd.set_option('display.max_colwidth', -1)


# Log - nginx

In [2]:
df_log = pd.read_csv('../data/api-georef-clean.csv', memory_map=True, index_col=0)

In [3]:
# print(df_log.shape)

uri_georef = df_log.uri.str.contains('/georef/api/')
status_ok = df_log.status == 200
status_nan = df_log.status.isnull()

df_log = df_log[uri_georef & ~status_nan & status_ok]

# separo columna endpoint
df_log['endpoint'] = df_log.uri.apply(lambda x: str(x).strip('/').split('/')[-1].split('.')[0])

# reemplazo los methods que no son GET o POST por OTROS
other_mr = ~df_log.method.isin(['GET','POST'])
df_log.loc[other_mr,'method'] = 'OTROS'

# agrego fecha corta
df_log['date_short'] = pd.DatetimeIndex(df_log.time_local).strftime('%Y-%m')

# agrego clasificación de qs's
qs_contains_name = df_log.querystring.str.contains('nombre=').astype(bool)
qs_contains_dir = df_log.querystring.str.contains('direccion=').astype(bool)
uri_is_ubicacion = df_log.uri.str.contains('ubicacion').astype(bool)

query_is_norm = (qs_contains_name & (~qs_contains_dir) & (~uri_is_ubicacion))
query_is_norm_dir = (qs_contains_dir & (~qs_contains_name) & (~uri_is_ubicacion))
query_is_ref = ~(query_is_norm | query_is_norm_dir | uri_is_ubicacion)

df_log.loc[uri_is_ubicacion, 'query_tipo'] = 'enriquecimiento'
df_log.loc[query_is_norm, 'query_tipo'] = 'normalizacion_nombres'
df_log.loc[query_is_norm_dir, 'query_tipo'] = 'normalizacion_direcciones'
df_log.loc[query_is_ref, 'query_tipo'] = 'referenciación'

# print(df_log.shape)

In [4]:
log_date_max = df_log.time_local.max()

log_date_min = df_log.time_local.min()

## Hits

In [5]:
df_hits = df_log.groupby(['method','endpoint','query_tipo'])['-'].count()

df_hits = df_hits.to_frame().reset_index().rename({'-':'hits'}, axis=1)


In [7]:
# df_hits

### Por método

### Por tipo 

In [8]:
# df_aux = df_log.dropna(axis='index', subset=['querystring'])


# uri_is_ubicacion = df_aux.uri.str.contains('ubicacion').astype(bool)

# qs_contains_name = df_aux.querystring.str.contains('nombre=').astype(bool)
# qs_contains_dir = df_aux.querystring.str.contains('direccion=').astype(bool)


# query_is_norm = (qs_contains_name & (~qs_contains_dir) & (~uri_is_ubicacion))
# query_is_norm_dir = (qs_contains_dir & (~qs_contains_name) & (~uri_is_ubicacion))
# query_is_enr = uri_is_ubicacion
# query_is_ref = ~(query_is_norm | query_is_norm_dir | query_is_enr)

# list_query_is = [query_is_norm, query_is_norm_dir, query_is_ref, query_is_enr]
# list_query_types = ['normalización de nombres', 'normalización de direcciones', 'referenciación', 'enriquecimiento']

# list_query_sum = [f.sum() for f in list_query_is]
# df_aux_ = pd.DataFrame({'hits':list_query_sum}, index=list_query_types)

In [9]:
# assert df_aux_.hits.sum() == df_log.dropna(axis='index', subset=['querystring']).shape[0]

In [10]:
# df_aux_

In [11]:
# df_aux[query_is_norm_dir].method.value_counts()

In [12]:
# del df_aux_

## Operaciones 

### Direcciones normalizadas

In [2]:
# Tomo sólo las querys de normalización de direcciones
# # df_aux = df_log.dropna(axis='index', subset=['querystring'])
# # df_aux = df_aux[query_is_norm_dir]

# df_aux = df_log.dropna(axis='index', subset=['querystring'])[query_is_norm_dir]
# df_aux = df_log.dropna(axis='index', subset=['querystring'])

# df_aux.body_bytes_sent.describe().to_frame()

In [3]:
# df_aux.body_bytes_sent.hist(bins=50, figsize=[18,6])
# plt.yscale('log')

In [4]:
# # cantidad de registros con body_bytes_sent mayor al min/a la media para una dirección
# min_one_direction = 622
# mean_one_direction = 678
# # (df_aux.body_bytes_sent > min_one_direction).sum()
# minbbs_rows_gt_mean = (df_aux.body_bytes_sent > mean_one_direction).sum()

In [5]:
# # registros con body_bytes_sent mayor al min/a la media para 500 direcciones
# min_max_directions = 311000
# mean_max_directions = 3344312
# maxbbs_rows_gt_min = (df_aux.body_bytes_sent > min_max_directions).sum()
# # (df_aux.body_bytes_sent > mean_max_directions).sum()

#### Pruebo una regresión lineal


In [6]:
# from sklearn.externals import joblib

# # Load the pickle file
# clf_load = joblib.load('saved_model.pkl') 

In [18]:
# df_aux['cantidad_pred'] = clf_load.predict(df_aux.body_bytes_sent.values.reshape(-1, 1))

In [19]:
# df_aux.cantidad_pred.describe()

In [20]:
# df_aux.cantidad_pred.hist(bins=50, figsize=[18,6])
# plt.yscale('log')
# plt.ylabel('')



# Residuales del log de SdT

In [21]:
# interest_domains = ['cnvivienda.org.ar',
#                    'andino.pilar.gov.ar',
#                    'johnny-tests.hopto.org',
#                    'datos.regionglobal.com.ar',
#                    'www.w3schools.com',
#                    'wonderful-gates-af1c3e.netlify.com',
#                    'translate.googleusercontent.com',
#                    'pentasistemas.com.ar',
#                    'www.blogger.com',
#                    'twitter.com',
#                    'github.com',
#                    'org.telegram.messenger',
#                    'run.scrimba.com',
#                    'www.translatoruser-int.com']

In [22]:
# is_interest_domain = df_log.domain.isin(interest_domains)
# # 
# df_log[is_interest_domain].groupby(['domain','path']).count()

# OSDE

In [21]:
# df_log.columns

In [23]:
df_osde_by_month = df_log[df_log.http_referer.str.contains('osde')].time_local.apply(lambda x: str(x)[:7]).value_counts().to_frame().sort_index()